In [2]:
import os
import sys
import src

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans'

In [5]:
import zipfile
import gdown
from pathlib import Path
from dataclasses import dataclass
from typing import Any
from box import ConfigBox
from src.common import read_yaml

In [19]:
# this goes to entity
@dataclass
class DataIntegrationConfig:
    root_dir: Path
    source_url: str
    zip_path: Path
    extract_path: Path
    
# now i have to pick the read the data and return the data integration object
    
# the job of this class is to return the config.yaml parameters
class ConfigurationManager:
    def __init__(self, config_path: Path, params_path: Path):
        self.config_path = config_path
        self.params_path = params_path
        
        self.config = read_yaml(self.config_path)
        self.params = read_yaml(self.params_path)
        
    
    def get_data_ingestion_prams(self) -> ConfigBox:
        config = self.config.data_ingestion
        
        return DataIntegrationConfig(
            root_dir= config.root_dir, 
            source_url = config.source_url,
            zip_path = config.zip_path,
            extract_path = config.extract_path
        )


In [20]:
os.getcwd()


'/Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans'

In [21]:
config_path = Path("config.yaml")
params_path = Path("params.yaml")
ob = ConfigurationManager(config_path, params_path)
data = ob.get_data_ingestion_prams()

In [22]:
from src import logger

class dataIngestion:
    def __init__(self, config: dict):
        self.config = config
    
    def download_data(self):
        try:
            dataset_url = self.config.source_url
            zip_path = self.config.zip_path
            extract_path = self.config.extract_path
            logger.info(f"Downloading dataset from url: {dataset_url}")

            os.makedirs("data/raw", exist_ok=True)
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,output = zip_path)
            logger.info(f"Dataset Downloaded to path: {zip_path}")
        except Exception as e:
            logger.info(e)

    def extract_data(self):
        try:
            extract_path = self.config.extract_path
            with zipfile.ZipFile(self.config.zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            logger.info(f'Data is extracted to path: {extract_path}')
        except Exception as e:
            logger.exception(e)
            
            

In [23]:
ingestion = dataIngestion(data)

In [24]:
ingestion.download_data()

Downloading...
From: https://drive.google.com/uc?/export=download&id=1ZF74F8h_419Lf-jK-k9TRMHY1GazwSQd
To: /Users/goldyrana/work/dl/kidney_scans/end_to_end_dl_project_kidney_scans/data/raw/data.zip
100%|██████████| 6.81M/6.81M [00:01<00:00, 4.70MB/s]


In [89]:
ingestion.extract_data()